In [8]:
import pickle

#data = [df,spray,(weather_st1, weather_st2)]

with open('cleanData.pickle', 'rb') as f:
    data = pickle.load(f)
    
df = data[0]

In [9]:
import shapefile
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd

In [10]:
from os import listdir
from os.path import isfile, join

parkDir = './AddData/Parks/'
waterDir = './AddData/Water/'

parks = [f for f in listdir(parkDir) if isfile(join(parkDir,f)) if f.count('.csv') ==0]
water = [f for f in listdir(waterDir) if isfile(join(waterDir,f))if f.count('.csv') ==0]

parkShape = parkDir + parks[0].split('.')[0]
waterShape = waterDir + water[0].split('.')[0]

In [11]:
psf = shapefile.Reader(parkShape)
wsf = shapefile.Reader(waterShape)

parkSR = psf.shapeRecords()
waterSR = wsf.shapeRecords()

In [19]:
parkSR[0].record[19] ## Area of park

1.14

In [20]:
parkSize = {}
for s in parkSR:
    parkSize[s.record[4]] = s.record[19]

In [12]:
# Create cKDTree functions in Dict

# Key: Identifier (number for water, park name for park)
# Value: cKDTree function built on all the points associated with water/park feature

waterFinder={}
for i, s in enumerate(waterSR):
    waterFinder[i] = cKDTree(s.shape.points)

parkFinder = {}
for s in parkSR:
    parkFinder[s.record[4]] = cKDTree(s.shape.points)

In [22]:
# Builds Distance Dictionaries:
# Key 1: Unique Location from DF
# Key 2: water/park ID
# Value: Distance from K1 to K2

uniqueLocs = df['Location'].unique()
parkDist = {}
waterDist = {}

for l in uniqueLocs:
    parkDist[l] = {}
    waterDist[l] = {}
    
    for k in parkFinder:
        parkDist[l][k]= (parkFinder[k].query(l,1)[0], parkSize[k])
    
    for k in waterFinder:
        waterDist[l][k] = waterFinder[k].query(l,1)[0]

In [44]:
# Dictionary For water features where:
# Key: Location
# Value: Distance to nearest water feature

distToWater = {}
for k in waterDist:
    distToWater[k] = min(waterDist[k].values())

# Dictionary for park features where:
# Key: Unique Location
# Value: Sum of proximity influence of all parks
# Proximity influence based on inverse square law:
# sum of all [ area of park / (distance to park)^2]

parkProx = {}

for k in parkDist:
    proxVal = 0
    for k2 in parkDist[k]:
        current = parkDist[k][k2]
        #print(current[1])
        #print(current[0]**2)
        proxVal += current[1] / ((1*current[0])**2)
    
    parkProx[k] = proxVal